In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')

from src._utils import load_bertopic_model_from_hf

In [2]:
import json
import requests

### Missing embeddings

In [3]:
from src.articles.ArticleService import ArticleService

In [4]:
a = ArticleService.get_Article('65a49fcfea435f28caee4d67')

2024-01-15 09:13:47,689| ERROR   | Problem setting SSH Forwarder up: Couldn't open tunnel 127.0.0.1:27017 <> insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com:27017 might be in use or destination not reachable


Exception: Error in _get_collection: Error in connecting to db: Failed to connect to MongoDB: An error occurred while opening tunnels.

In [3]:
from sql.articles.ArticleAttributesSQL import ArticleAttributesSQL

In [4]:
att = ArticleAttributesSQL.get_attributes_for_article_id('65a49fcfea435f28caee4d67')

In [5]:
att

{'65a49fcfea435f28caee4d67': {'financial_news': True,
  'relevant_for_india': True,
  'validity_duration': 1,
  'expected_popularity': 'niche'}}

In [ ]:
from application import evaluate_save_article_to_candidates

In [32]:
len(results)

1601

In [17]:
r = requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/assign_cluster', json = {'articleId': '65a49fcfea435f28caee4d67'})

In [18]:
r.text

'cluster successfully assigned for articleId 65a49fcfea435f28caee4d67'

In [10]:
# r = requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/get_missing_candidate_summaries')

In [287]:
# r = requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/update_candidates')

In [30]:
k = json.loads(r.text)

In [31]:
recent_articles = k['articleIds']

In [32]:
len(recent_articles)

770

In [33]:
from datetime import datetime, timedelta, timezone

In [36]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [37]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT DISTINCT article_id FROM embeddings'
    cursor.execute(sql)
    results = cursor.fetchall()
embeddings_articles = [x[0] for x in results]

In [38]:
missing_articles = set(recent_articles) - set(embeddings_articles)

missing_articles = list(missing_articles)

In [39]:
len(missing_articles)

0

In [314]:
a,b = get_all_relevant_embeddings()

In [254]:
num_chunks = len(missing_articles)//100 + 1

In [20]:
def create_missing_article_embedding(article_id):
    r = requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/create_embedding', json={'articleId': article_id})
    if r.status_code == 200:
        return article_id

In [16]:
r = requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/create_embedding', json={'articleId': '659faf5fe362d630b647262a'})

In [18]:
podcast_episodes = ['659faf5fe362d630b647262a', '659faf60e362d630b647262b', '659faf61e362d630b647262e', '659faf61e362d630b647262d', 
                    '659faf64e362d630b6472634', '659faf63e362d630b6472632', '659faf65e362d630b6472635', '659faf64e362d630b6472633',
                   '659faf63e362d630b6472631', '659faf60e362d630b647262c', '659faf66e362d630b6472638', '659faf62e362d630b6472630',
                   '659faf65e362d630b6472636', '659faf66e362d630b6472637', '659faf67e362d630b6472639', '659faf62e362d630b647262f', 
                   '659fb612e362d630b647263a', '659fb615e362d630b647263f', '659fb616e362d630b6472640', '659fb615e362d630b647263e', 
                   '659fb617e362d630b6472642', '659fb613e362d630b647263b', '659fb614e362d630b647263c']

In [22]:
len(podcast_episodes)

23

In [21]:
for art in podcast_episodes:
    create_missing_article_embedding(art)

In [23]:
def create_podcast_attributes(article_id):
    r = requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm', json={'articleId': article_id})
    if r.status_code == 200:
        return article_id

In [25]:
for art in podcast_episodes:
    create_podcast_attributes(art)

In [28]:
requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm', json={'articleId': '659fca41e362d630b647264a'})

<Response [200]>

In [98]:
requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm', json={'articleId': '653680261e5cc42b1b144145'})

<Response [200]>

In [17]:
r.text

"embeddings created for articleId 659faf5fe362d630b647262a and got response of {'MD5OfMessageBody': '7b28515ef1a434131c204debe17c3ee1', 'MessageId': '10b99120-d845-445c-afec-bd7dc5e63ea9', 'ResponseMetadata': {'RequestId': '92e52317-da82-5bd5-984c-7d0e23fdc7db', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '92e52317-da82-5bd5-984c-7d0e23fdc7db', 'date': 'Thu, 11 Jan 2024 14:12:15 GMT', 'content-type': 'text/xml', 'content-length': '378', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}"

In [245]:
create_missing_article_embedding(some_article)

'657aa8823a7a250d32265353'

In [239]:
import time

In [255]:
completed_article_ids = []

In [256]:
for i in range(num_chunks):
    article_chunk = missing_articles[i*100:(i+1)*100]
    print(f'starting creation at {"{:%b %d, %Y %H:%M}".format(datetime.now())}')
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(create_missing_article_embedding, art_id) for art_id in article_chunk]

        for future in as_completed(futures):
            response = future.result()
            if response:  # Timeout or other errors
                completed_article_ids.append(response)
                continue
    print(f'completed chunk {(i + 1) * 100} articles at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

starting creation at Dec 14, 2023 14:18
completed chunk 100 articles at Dec 14, 2023 14:22 in 4.466666666666667 minutes


### Missing cluster mapping

In [316]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT DISTINCT article_id FROM embeddings'
    cursor.execute(sql)
    results = cursor.fetchall()
embeddings_articles = [x[0] for x in results]

In [317]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT DISTINCT article_id FROM article_to_cluster_mapping'
    cursor.execute(sql)
    results = cursor.fetchall()
clustered_articles = [x[0] for x in results]

In [318]:
unclustered_articles = list(set(recent_articles) - set(clustered_articles))

In [319]:
len(unclustered_articles)

561

In [324]:
r = requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/update_candidates')

In [325]:
r.text

'candidates updated'

In [19]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [31]:
with PostgresDatabaseOperation() as cursor:
    sql = "SELECT article_id FROM embeddings e WHERE e.created_at > '2024-01-01' AND article_id NOT IN (SELECT article_id FROM article_to_cluster_mapping)"
    cursor.execute(sql)
    results = cursor.fetchall()

In [35]:
unclustered_articles = [x[0] for x in results]

In [33]:
def assign_missing_article_clusters(article_id):
    r = requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/assign_cluster', json={'articleId': article_id})
    if r.status_code == 200:
        return article_id

In [37]:
num_chunks = len(unclustered_articles)//50 + 1

In [38]:
completed_article_ids = []

In [42]:
from datetime import datetime
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

In [43]:
for i in range(num_chunks):
    article_chunk = unclustered_articles[i*50:(i+1)*50]
    print(f'starting creation at {"{:%b %d, %Y %H:%M}".format(datetime.now())}')
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(assign_missing_article_clusters, art_id) for art_id in article_chunk]

        for future in as_completed(futures):
            response = future.result()
            if response:  # Timeout or other errors
                completed_article_ids.append(response)
                continue
    print(f'completed chunk {(i + 1) * 50} articles at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

starting creation at Jan 15, 2024 10:56
completed chunk 50 articles at Jan 15, 2024 10:56 in 0.15 minutes
starting creation at Jan 15, 2024 10:56
completed chunk 100 articles at Jan 15, 2024 10:57 in 0.16666666666666666 minutes
starting creation at Jan 15, 2024 10:57
completed chunk 150 articles at Jan 15, 2024 10:57 in 0.13333333333333333 minutes
starting creation at Jan 15, 2024 10:57
completed chunk 200 articles at Jan 15, 2024 10:57 in 0.15 minutes
starting creation at Jan 15, 2024 10:57
completed chunk 250 articles at Jan 15, 2024 10:57 in 0.15 minutes
starting creation at Jan 15, 2024 10:57
completed chunk 300 articles at Jan 15, 2024 10:57 in 0.16666666666666666 minutes
starting creation at Jan 15, 2024 10:57
completed chunk 350 articles at Jan 15, 2024 10:57 in 0.13333333333333333 minutes
starting creation at Jan 15, 2024 10:57
completed chunk 400 articles at Jan 15, 2024 10:58 in 0.15 minutes
starting creation at Jan 15, 2024 10:58
completed chunk 450 articles at Jan 15, 2024 

#### LLM attributes for missing articles

In [95]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT DISTINCT article_id FROM llm_article_attributes'
    cursor.execute(sql)
    results = cursor.fetchall()
llm_attribute_articles = [x[0] for x in results]

In [96]:
pending_attributes_articles = list(set(recent_articles) - set(llm_attribute_articles))

In [97]:
len(pending_attributes_articles)

1

In [50]:
def get_missing_article_attributes(article_id):
    r = requests.post('http://Arcane-env.eba-mrsaixmg.ap-south-1.elasticbeanstalk.com/get_article_attributes_from_llm', json={'articleId': article_id})
    if r.status_code == 200:
        return article_id

In [83]:
num_chunks = len(pending_attributes_articles)//20 + 1

In [84]:
completed_article_ids = []

In [85]:
import time

In [86]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [88]:
for i in range(num_chunks):
    article_chunk = pending_attributes_articles[i*20:(i+1)*20]
    print(f'starting creation at {"{:%b %d, %Y %H:%M}".format(datetime.now())}')
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_missing_article_attributes, art_id) for art_id in article_chunk]

        for future in as_completed(futures):
            response = future.result()
            if response:  # Timeout or other errors
                completed_article_ids.append(response)
                continue
    print(f'completed chunk {(i + 1) * 100} articles at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

starting creation at Jan 12, 2024 11:00
completed chunk 100 articles at Jan 12, 2024 11:01 in 1.0 minutes
